In [2]:
import requests

In [30]:
url = "https://roboats.virtualregatta.com/api/"

def post_login() :
    
    r = requests.post(url+"login",
                  json={"email" : "malo.adler@telecom-paris.fr",
                            "password" : "wonderduck",
                            "raceId" : 531,
                            "legNum" : 1} )
    
    if(r.status_code != 200) :
        raise SystemExit("Login failed : " + str(r.status_code))
        
    return(r.headers['token'])

############################################################################################################################

def get_info_slow(token) :
    
    r = requests.get(url+"infos/slow",
                  headers={"token" : token })
    
    if(r.status_code != 200) :
        raise SystemExit("Get info slow failed : " + str(r.status_code))
        
    return(r.text)

############################################################################################################################
def get_info_fast(token) :
    
    r = requests.get(url+"infos/fast",
                  headers={"token" : token })
    
    if(r.status_code != 200) :
        raise SystemExit("Get info fast failed : " + str(r.status_code))
        
    return(r.text)

############################################################################################################################

def put_boat_action(token , json) :
    
    r = requests.put(url+"boat/actions",
                    headers={"token" : token },
                    json=json)
    
    if(r.status_code != 200) :
        raise SystemExit("Put boat action failed : " + str(r.status_code))
        
    return(r)

############################################################################################################################

def change_heading(token , deg , lock_sail) :
    
    if deg < 0 or deg > 360 :
        raise SystemExit("Degree should be between 0 and 360")
        
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "heading",
                "values" : {
                "deg": deg,
                "autoTwa": lock_sail
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)

############################################################################################################################

def change_sail(token , sailId) :
    
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "sail",
                "values" : {
                    "sailId" : sailId
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)
    
############################################################################################################################

def place_waypoint(token , wpid , lon , lat) :
    
    json = {
        "tsLegStart": 1640010600000,
        "actions" : [
            {
                "type": "wp",
                "values" : {
                    "nextWpIdx": wpid + 1,
                    "coordinates": [
                        {
                            "idx": wpid,
                            "pos" : {
                                "lon": lon,
                                "lat": lat
                            }
                        }
                    ]
                }
            }
        ]
    }
    
    r = put_boat_action(token , json)

In [28]:
token = post_login()
change_heading(token , 56 , False)
change_sail(token , 11)
info_slow = get_info_slow(token)
info_fast = get_info_fast(token)
#place_waypoint(token , 1 , 48 , -7)

In [29]:
info_fast

'{"rc":"ok","res":{"ba":[{"timestamp":1652630940000,"type":"heading","values":{"deg":56,"autoTwa":false}},{"timestamp":1652630940000,"type":"sail","values":{"sailId":11}}],"bs":{"aground":false,"badSail":false,"distanceFromStart":8.0078,"distanceToEnd":3326.7389,"gateGroupCounters":[0],"hasPermanentAutoSails":true,"heading":51,"icelimit":false,"isRegulated":false,"polar":4,"pos":{"lat":48.162909,"lon":-4.939376},"rank":0,"sail":17,"speed":1.6952,"stamina":null,"state":"racing","twa":-171.82697,"twaAuto":51,"twd":239.17303,"tws":8.55262,"tsEndOfAutoSail":1738944480000,"tsEndOfGybe":1652631240000,"tsEndOfSailChange":1652631300000,"tsEndOfTack":1652630880000,"tsLastCalc":1652630880000},"engine":{"tsLastCalc":1652630880000,"tsNextCalc":1652631030000}}}'